In [1]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule

import hdnntools as hdt

import numpy as np
import  ase
#from ase.build import molecule
#from ase.neb import NEB
#from ase.calculators.mopac import MOPAC
from ase.md.langevin import Langevin
from ase.md.verlet import VelocityVerlet
from ase.io.trajectory import Trajectory
from ase import units

from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

from ase.optimize.fire import FIRE as QuasiNewton

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

#from ase.neb import NEBtools
from ase.io import read, write
from ase.optimize import BFGS, LBFGS

import time

van Der Waals correction will be unavailable. Please install ased3


In [2]:
dir = '/scratch/Research/gutzwiller_research/gutz_new_data/ML-GQMD-data-NEW/case1/train_models/data-U0.001/testset/'

# Molecule file
molfile = '/nh/nest/u/jsmith/scratch/Research/gutzwiller_research/gutz_new_data/ML-GQMD-data-NEW/case1/test_files/data-U0.001/ts0/sys-000.xyz'

# Dynamics file
xyzfile = dir + 'mdcrd.xyz'

# Trajectory file
trajfile = dir + 'traj.dat'

# Optimized structure out:
optfile = dir + 'optmol.xyz'

ntdir = '/scratch/Research/gutzwiller_research/gutz_new_data/ML-GQMD-data-NEW/case1/train_models/data-U0.001/testsets/testset0'

T = 100.0 # Temperature
dt = 0.1
C = 2.0 # Optimization convergence
steps = 40000

ntdir = '/scratch/Research/gutzwiller_research/gutz_new_data/ML-GQMD-data-NEW/case1/train_models/data-U0.001/'
cns = ntdir + '../../params/rX-2.5R_32-2.0A_a8-8.params'
sae = ntdir + '../../params/sae.dat'
nnf = ntdir + 'train'
Nn = 3

#ntdir = '/nh/nest/u/jsmith/Research/gutzwiller_research/DarwinSets/GutzTrain/train_test_U2/U2_co3/'
#cns = ntdir + 'rX-6.0A_32-4.0A_a8-8.params'
#sae = ntdir + 'sae.dat'
#nnf = ntdir + 'train'
#Nn = 5

In [3]:
# Load molecule
mol = read(molfile)
L = 10.9753
mol.set_cell(([[L, 0, 0],
               [0, L, 0],
               [0, 0, L]]))

mol.set_pbc((True, True, True))

In [4]:
# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 1)

# Set ANI calculator
mol.set_calculator(ANIENS(aens,sdmx=20000000.0))

In [5]:
def erg_exp (Rmag):
    return 100.0*np.exp(-3.53*Rmag)

def frc_exp (Rmag,R):
    F = -3.53*erg_exp(Rmag)*(R/Rmag)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
    return F

mol.calc.set_pairwise(erg_exp, frc_exp)

In [6]:
# Optimize molecule
start_time = time.time()
dyn = QuasiNewton(mol)
dyn.run(fmax=C)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

# Save optimized mol
spc = mol.get_chemical_symbols()
pos = mol.get_positions(wrap=False).reshape(1,len(spc),3)

hdt.writexyzfile(optfile, pos, spc)

      Step     Time          Energy         fmax
FIRE:    0 17:01:04      549.643799       30.9240
FIRE:    1 17:01:04      503.265288       25.9490
FIRE:    2 17:01:04      461.955264       21.8988
FIRE:    3 17:01:04      425.120900       18.5820
FIRE:    4 17:01:04      392.252456       15.8506
FIRE:    5 17:01:04      362.909092       13.5893
FIRE:    6 17:01:04      336.707389       11.7079
FIRE:    7 17:01:04      313.311994       10.1353
FIRE:    8 17:01:04      292.426402        8.8162
FIRE:    9 17:01:04      273.788613        7.7057
FIRE:   10 17:01:04      257.165546        6.7674
FIRE:   11 17:01:04      242.348471        5.9718
FIRE:   12 17:01:05      229.149163        5.3509
FIRE:   13 17:01:05      217.396724        4.9769
FIRE:   14 17:01:05      206.934968        4.6953
FIRE:   15 17:01:05      197.620336        4.4344
FIRE:   16 17:01:05      189.320378        4.2262
FIRE:   17 17:01:05      181.912702        4.0278
FIRE:   18 17:01:05      175.284507        3.8293
F

In [7]:
MaxwellBoltzmannDistribution(mol, 10000.0 * units.kB)

# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = VelocityVerlet(mol, dt * units.fs)

In [8]:
# Open MD output
mdcrd = open(xyzfile,'w')

# Open MD output
traj = open(trajfile,'w')

# Define the printer
def storeenergy(a=mol, d=dyn, b=mdcrd, t=traj):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)

    stddev =  a.calc.stddev

    t.write(str(d.get_number_of_steps()) + ' ' + str(ekin / (1.5 * units.kB)) + ' ' + str(epot) + ' ' + str(ekin) + ' ' + str(epot+ekin) + '\n')
    b.write(str(len(a)) + '\n' + str(ekin / (1.5 * units.kB)) + ' Step: ' + str(d.get_number_of_steps()) + '\n')
    c = a.get_positions(wrap=True)
    for j, i in zip(a, c):
        b.write(str('C') + ' ' + str(i[0]) + ' ' + str(i[1]) + ' ' + str(i[2]) + '\n')

    print('Step: %d Energy per atom: Epot = %.3f  Ekin = %.3f (T=%.3fK)  '
          'Etot = %.6f' ' StdDev = %.3f' % (d.get_number_of_steps(), epot, ekin, ekin / (1.5 * units.kB), epot + ekin, stddev))
dyn.attach(storeenergy, interval=1)

In [9]:
start_time = time.time()
dyn.run(1000000)
print('[ANI Total time:', time.time() - start_time, 'seconds]')


Step: 1 Energy per atom: Epot = 0.632  Ekin = 1.338 (T=10351.658K)  Etot = 1.970306 StdDev = 814.555
Step: 2 Energy per atom: Epot = 0.634  Ekin = 1.337 (T=10341.639K)  Etot = 1.970309 StdDev = 814.813
Step: 3 Energy per atom: Epot = 0.636  Ekin = 1.335 (T=10324.807K)  Etot = 1.970310 StdDev = 815.105
Step: 4 Energy per atom: Epot = 0.639  Ekin = 1.332 (T=10301.290K)  Etot = 1.970311 StdDev = 815.401
Step: 5 Energy per atom: Epot = 0.643  Ekin = 1.328 (T=10271.224K)  Etot = 1.970312 StdDev = 815.719
Step: 6 Energy per atom: Epot = 0.647  Ekin = 1.323 (T=10234.755K)  Etot = 1.970311 StdDev = 816.086
Step: 7 Energy per atom: Epot = 0.653  Ekin = 1.317 (T=10192.036K)  Etot = 1.970310 StdDev = 816.516
Step: 8 Energy per atom: Epot = 0.659  Ekin = 1.311 (T=10143.230K)  Etot = 1.970310 StdDev = 816.997
Step: 9 Energy per atom: Epot = 0.666  Ekin = 1.304 (T=10088.510K)  Etot = 1.970309 StdDev = 817.491
Step: 10 Energy per atom: Epot = 0.674  Ekin = 1.296 (T=10028.063K)  Etot = 1.970309 StdDev

Step: 83 Energy per atom: Epot = 0.856  Ekin = 1.115 (T=8624.558K)  Etot = 1.970506 StdDev = 823.825
Step: 84 Energy per atom: Epot = 0.858  Ekin = 1.113 (T=8609.021K)  Etot = 1.970509 StdDev = 824.044
Step: 85 Energy per atom: Epot = 0.860  Ekin = 1.111 (T=8592.425K)  Etot = 1.970511 StdDev = 824.245
Step: 86 Energy per atom: Epot = 0.862  Ekin = 1.108 (T=8575.039K)  Etot = 1.970514 StdDev = 824.446
Step: 87 Energy per atom: Epot = 0.864  Ekin = 1.106 (T=8557.102K)  Etot = 1.970517 StdDev = 824.620
Step: 88 Energy per atom: Epot = 0.867  Ekin = 1.104 (T=8538.819K)  Etot = 1.970520 StdDev = 824.732
Step: 89 Energy per atom: Epot = 0.869  Ekin = 1.101 (T=8520.362K)  Etot = 1.970524 StdDev = 824.780
Step: 90 Energy per atom: Epot = 0.872  Ekin = 1.099 (T=8501.869K)  Etot = 1.970527 StdDev = 824.803
Step: 91 Energy per atom: Epot = 0.874  Ekin = 1.097 (T=8483.450K)  Etot = 1.970530 StdDev = 824.864
Step: 92 Energy per atom: Epot = 0.876  Ekin = 1.094 (T=8465.191K)  Etot = 1.970533 StdDev 

Step: 166 Energy per atom: Epot = 0.895  Ekin = 1.075 (T=8319.661K)  Etot = 1.970503 StdDev = 827.890
Step: 167 Energy per atom: Epot = 0.892  Ekin = 1.078 (T=8340.475K)  Etot = 1.970496 StdDev = 828.008
Step: 168 Energy per atom: Epot = 0.890  Ekin = 1.081 (T=8360.312K)  Etot = 1.970490 StdDev = 828.113
Step: 169 Energy per atom: Epot = 0.887  Ekin = 1.083 (T=8379.049K)  Etot = 1.970483 StdDev = 828.156
Step: 170 Energy per atom: Epot = 0.885  Ekin = 1.085 (T=8396.594K)  Etot = 1.970478 StdDev = 828.108
Step: 171 Energy per atom: Epot = 0.883  Ekin = 1.087 (T=8412.886K)  Etot = 1.970472 StdDev = 827.967
Step: 172 Energy per atom: Epot = 0.881  Ekin = 1.089 (T=8427.890K)  Etot = 1.970467 StdDev = 827.754
Step: 173 Energy per atom: Epot = 0.879  Ekin = 1.091 (T=8441.597K)  Etot = 1.970462 StdDev = 827.495
Step: 174 Energy per atom: Epot = 0.878  Ekin = 1.093 (T=8454.017K)  Etot = 1.970457 StdDev = 827.204
Step: 175 Energy per atom: Epot = 0.876  Ekin = 1.094 (T=8465.176K)  Etot = 1.9704

Step: 248 Energy per atom: Epot = 0.926  Ekin = 1.045 (T=8082.585K)  Etot = 1.970510 StdDev = 829.038
Step: 249 Energy per atom: Epot = 0.923  Ekin = 1.048 (T=8104.168K)  Etot = 1.970503 StdDev = 828.960
Step: 250 Energy per atom: Epot = 0.920  Ekin = 1.050 (T=8126.625K)  Etot = 1.970496 StdDev = 828.879
Step: 251 Energy per atom: Epot = 0.917  Ekin = 1.053 (T=8149.691K)  Etot = 1.970489 StdDev = 828.794
Step: 252 Energy per atom: Epot = 0.914  Ekin = 1.056 (T=8173.108K)  Etot = 1.970482 StdDev = 828.675
Step: 253 Energy per atom: Epot = 0.911  Ekin = 1.059 (T=8196.633K)  Etot = 1.970475 StdDev = 828.480
Step: 254 Energy per atom: Epot = 0.908  Ekin = 1.063 (T=8220.040K)  Etot = 1.970468 StdDev = 828.189
Step: 255 Energy per atom: Epot = 0.905  Ekin = 1.066 (T=8243.128K)  Etot = 1.970462 StdDev = 827.824
Step: 256 Energy per atom: Epot = 0.902  Ekin = 1.068 (T=8265.716K)  Etot = 1.970455 StdDev = 827.425
Step: 257 Energy per atom: Epot = 0.899  Ekin = 1.071 (T=8287.651K)  Etot = 1.9704

Step: 331 Energy per atom: Epot = 0.864  Ekin = 1.107 (T=8562.489K)  Etot = 1.970369 StdDev = 826.286
Step: 332 Energy per atom: Epot = 0.865  Ekin = 1.106 (T=8554.594K)  Etot = 1.970365 StdDev = 826.364
Step: 333 Energy per atom: Epot = 0.866  Ekin = 1.105 (T=8546.761K)  Etot = 1.970360 StdDev = 826.332
Step: 334 Energy per atom: Epot = 0.867  Ekin = 1.104 (T=8539.037K)  Etot = 1.970356 StdDev = 826.201
Step: 335 Energy per atom: Epot = 0.868  Ekin = 1.103 (T=8531.458K)  Etot = 1.970351 StdDev = 826.016
Step: 336 Energy per atom: Epot = 0.869  Ekin = 1.102 (T=8524.049K)  Etot = 1.970347 StdDev = 825.832
Step: 337 Energy per atom: Epot = 0.869  Ekin = 1.101 (T=8516.828K)  Etot = 1.970342 StdDev = 825.693
Step: 338 Energy per atom: Epot = 0.870  Ekin = 1.100 (T=8509.809K)  Etot = 1.970338 StdDev = 825.613
Step: 339 Energy per atom: Epot = 0.871  Ekin = 1.099 (T=8502.998K)  Etot = 1.970334 StdDev = 825.587
Step: 340 Energy per atom: Epot = 0.872  Ekin = 1.098 (T=8496.402K)  Etot = 1.9703

...

Step: 10016 Energy per atom: Epot = 0.938  Ekin = 1.033 (T=7990.226K)  Etot = 1.970451 StdDev = 826.318
Step: 10017 Energy per atom: Epot = 0.937  Ekin = 1.033 (T=7993.527K)  Etot = 1.970446 StdDev = 826.448
Step: 10018 Energy per atom: Epot = 0.936  Ekin = 1.034 (T=7999.800K)  Etot = 1.970441 StdDev = 826.579
Step: 10019 Energy per atom: Epot = 0.935  Ekin = 1.035 (T=8008.869K)  Etot = 1.970434 StdDev = 826.700
Step: 10020 Energy per atom: Epot = 0.934  Ekin = 1.037 (T=8020.492K)  Etot = 1.970427 StdDev = 826.817
Step: 10021 Energy per atom: Epot = 0.932  Ekin = 1.039 (T=8034.376K)  Etot = 1.970420 StdDev = 826.935
Step: 10022 Energy per atom: Epot = 0.930  Ekin = 1.041 (T=8050.191K)  Etot = 1.970413 StdDev = 827.043
Step: 10023 Energy per atom: Epot = 0.928  Ekin = 1.043 (T=8067.588K)  Etot = 1.970406 StdDev = 827.110
Step: 10024 Energy per atom: Epot = 0.925  Ekin = 1.045 (T=8086.216K)  Etot = 1.970400 StdDev = 827.102
Step: 10025 Energy per atom: Epot = 0.923  Ekin = 1.048 (T=8105.

Step: 10098 Energy per atom: Epot = 0.890  Ekin = 1.081 (T=8359.354K)  Etot = 1.970405 StdDev = 825.238
Step: 10099 Energy per atom: Epot = 0.888  Ekin = 1.082 (T=8371.021K)  Etot = 1.970409 StdDev = 825.247
Step: 10100 Energy per atom: Epot = 0.887  Ekin = 1.083 (T=8381.031K)  Etot = 1.970413 StdDev = 825.302
Step: 10101 Energy per atom: Epot = 0.886  Ekin = 1.084 (T=8389.436K)  Etot = 1.970417 StdDev = 825.373
Step: 10102 Energy per atom: Epot = 0.885  Ekin = 1.085 (T=8396.294K)  Etot = 1.970421 StdDev = 825.439
Step: 10103 Energy per atom: Epot = 0.884  Ekin = 1.086 (T=8401.664K)  Etot = 1.970425 StdDev = 825.465
Step: 10104 Energy per atom: Epot = 0.884  Ekin = 1.087 (T=8405.599K)  Etot = 1.970428 StdDev = 825.427
Step: 10105 Energy per atom: Epot = 0.884  Ekin = 1.087 (T=8408.148K)  Etot = 1.970431 StdDev = 825.353
Step: 10106 Energy per atom: Epot = 0.883  Ekin = 1.087 (T=8409.349K)  Etot = 1.970434 StdDev = 825.322
Step: 10107 Energy per atom: Epot = 0.883  Ekin = 1.087 (T=8409.

Step: 10180 Energy per atom: Epot = 0.881  Ekin = 1.089 (T=8428.482K)  Etot = 1.970422 StdDev = 826.549
Step: 10181 Energy per atom: Epot = 0.881  Ekin = 1.090 (T=8428.875K)  Etot = 1.970421 StdDev = 826.611
Step: 10182 Energy per atom: Epot = 0.881  Ekin = 1.090 (T=8429.771K)  Etot = 1.970419 StdDev = 826.749
Step: 10183 Energy per atom: Epot = 0.881  Ekin = 1.090 (T=8431.300K)  Etot = 1.970417 StdDev = 826.948
Step: 10184 Energy per atom: Epot = 0.880  Ekin = 1.090 (T=8433.616K)  Etot = 1.970414 StdDev = 827.180
Step: 10185 Energy per atom: Epot = 0.880  Ekin = 1.091 (T=8436.892K)  Etot = 1.970411 StdDev = 827.393
Step: 10186 Energy per atom: Epot = 0.879  Ekin = 1.091 (T=8441.302K)  Etot = 1.970408 StdDev = 827.522
Step: 10187 Energy per atom: Epot = 0.879  Ekin = 1.092 (T=8447.018K)  Etot = 1.970404 StdDev = 827.523
Step: 10188 Energy per atom: Epot = 0.878  Ekin = 1.093 (T=8454.187K)  Etot = 1.970400 StdDev = 827.392
Step: 10189 Energy per atom: Epot = 0.876  Ekin = 1.094 (T=8462.

Step: 10262 Energy per atom: Epot = 0.835  Ekin = 1.135 (T=8784.218K)  Etot = 1.970460 StdDev = 826.393
Step: 10263 Energy per atom: Epot = 0.838  Ekin = 1.132 (T=8760.456K)  Etot = 1.970462 StdDev = 826.447
Step: 10264 Energy per atom: Epot = 0.841  Ekin = 1.129 (T=8734.737K)  Etot = 1.970465 StdDev = 826.539
Step: 10265 Energy per atom: Epot = 0.845  Ekin = 1.125 (T=8707.260K)  Etot = 1.970467 StdDev = 826.628
Step: 10266 Energy per atom: Epot = 0.849  Ekin = 1.122 (T=8678.250K)  Etot = 1.970470 StdDev = 826.698
Step: 10267 Energy per atom: Epot = 0.853  Ekin = 1.118 (T=8647.957K)  Etot = 1.970473 StdDev = 826.770
Step: 10268 Energy per atom: Epot = 0.857  Ekin = 1.114 (T=8616.645K)  Etot = 1.970475 StdDev = 826.850
Step: 10269 Energy per atom: Epot = 0.861  Ekin = 1.110 (T=8584.587K)  Etot = 1.970477 StdDev = 826.920
Step: 10270 Energy per atom: Epot = 0.865  Ekin = 1.105 (T=8552.053K)  Etot = 1.970479 StdDev = 826.976
Step: 10271 Energy per atom: Epot = 0.869  Ekin = 1.101 (T=8519.

KeyboardInterrupt: 